In [154]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import sys,os
sys.path.append(os.getcwd())
from descriptive_analysis import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Building an Optimal Premium Model in an insurance company #

## Problem description ##

We are interested in solving a CRM problem for an insurance company. The tasks to be achieved are:

* Finding the ideal target, in this case, people who are more likely to contract the insurance.
* Obtaining the premium we should offer to each client, it means, the optimal price that should be offered to the clients.
* Calculating the difference between offering the premium randomly and optimally using the information obtained in the model.


## Working with data ##

Two databases with clients’ information are available.

In the first one we have the information of 20.000 clients which have already been contacted; 9% of them have contracted the product.
 
Important data is included such as the premium offered, the number of products that they have already bought, the number of years that they have been clients of the company and the socioeconomic status (an economic and sociological measure combined with the person’s work experience and its individual’s or family’s economic and social position in relation to others, based on income, education, and occupation).

In the second database of non-previously contacted clients, we have the same information of 10.000 clients but only 5.000 are going to be contacted due to mechanical restrictions.

Is worthwhile offering the same premium to all the clients? Is better focusing on people with some characteristics than choosing the clients randomly? 

In [8]:
xls = pd.ExcelFile('Database.xlsx')
variable_description = xls.parse(0)
db1 = xls.parse(1)
db2 = xls.parse(2)

In [9]:
variable_description

,Variable Name,Meaning
0,Obs,Number of Observations
1,Sales,It indicates whether the client bought a produ...
2,Price Sensitivity,It indicates the client's sensitivity to the p...
3,PhoneType,Client's phone type: Fixed or Mobile
4,Email,It indicates whether the client's email is ava...
5,Tenure,Client's tenure (year when the person became a...
6,NumberofCampaigns,Number of times the client has been called
7,ProdActive,Number of active products
8,ProdBought,Number of different products previously bought
9,Premium Offered,Premium offered to the client


In [10]:
db1.describe()

/usr/local/lib/python3.6/site-packages/numpy/lib/function_base.py:4274: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


,Obs,Sales,Price Sensitivity,Email,Tenure,NumberofCampaigns,ProdActive,ProdBought,Premium Offered,Birthdate,...,Income,yearBuilt,House Insurance,Pension Plan,Credit,Savings,Number of Mobile Phones,Number of Fixed Lines,ADSL,3G Devices
count,20000.000000,20000.000000,1475.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,9512.000000,...,1.486000e+04,14865.000000,14860.000000,1.486000e+04,1.486000e+04,1.484500e+04,7164.000000,7164.000000,7164.000000,7164.000000
mean,10000.500000,0.085750,3.792542,0.074450,2007.118950,3.709200,0.050700,0.320200,13.831877,1966.409062,...,5.802174e+04,1979.502657,7364.270664,3.728715e+04,1.696786e+04,3.055875e+04,1.504467,1.005444,0.502233,0.503769
std,5773.647028,0.280002,1.694535,0.262508,6.715032,4.156429,0.238186,0.706397,2.774808,11.478364,...,6.644060e+04,23.073381,8542.363258,4.239543e+04,1.945761e+04,7.872496e+04,1.120473,0.817475,0.500030,0.500021
min,1.000000,0.000000,1.000000,0.000000,1990.000000,2.000000,0.000000,0.000000,11.120000,1944.000000,...,2.190805e+03,1900.000000,186.013800,1.661723e+03,6.173887e+02,0.000000e+00,0.000000,0.000000,0.000000,0.000000
25%,5000.750000,0.000000,NaN,0.000000,2004.000000,2.000000,0.000000,0.000000,11.120000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,10000.500000,0.000000,NaN,0.000000,2010.000000,2.000000,0.000000,0.000000,14.500000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,15000.250000,0.000000,NaN,0.000000,2012.000000,4.000000,0.000000,0.000000,14.500000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,20000.000000,1.000000,6.000000,1.000000,2013.000000,32.000000,3.000000,6.000000,21.850000,1984.000000,...,4.106372e+06,2012.000000,527866.417800,2.620520e+06,1.202556e+06,4.884174e+06,3.000000,2.000000,1.000000,1.000000


In [11]:
db2.describe()

/usr/local/lib/python3.6/site-packages/numpy/lib/function_base.py:4274: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


,Obs,Price Sensitivity,Email,Tenure,NumberofCampaigns,ProdActive,ProdBought,Birthdate,Living Area (m^2),House Price,Income,yearBuilt,House Insurance,Pension Plan,Credit,Savings,Number of Mobile Phones,Number of Fixed Lines,ADSL,3G Devices
count,10000.00000,2694.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,4541.000000,7831.000000,7.830000e+03,7.831000e+03,7831.000000,7831.000000,7.831000e+03,7.831000e+03,7.831000e+03,3760.000000,3760.000000,3760.000000,3760.000000
mean,5000.50000,3.399777,0.046900,2011.248300,3.571400,0.022300,0.24210,1966.269764,203.416630,2.198855e+05,6.002617e+04,1978.753288,7648.644968,3.849267e+04,1.756162e+04,3.229642e+04,1.520479,0.992287,0.509309,0.499202
std,2886.89568,1.766382,0.211435,5.019897,4.237747,0.156221,0.64802,11.597210,597.405984,3.112247e+05,8.546163e+04,23.229961,10987.924223,5.453266e+04,2.502805e+04,8.328343e+04,1.116566,0.813359,0.499980,0.500066
min,1.00000,1.000000,0.000000,1950.000000,2.000000,0.000000,0.00000,1944.000000,0.000000,7.057584e+03,1.580315e+03,1900.000000,134.177400,1.198663e+03,4.453454e+02,0.000000e+00,0.000000,0.000000,0.000000,0.000000
25%,2500.75000,NaN,0.000000,2012.000000,2.000000,0.000000,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,5000.50000,NaN,0.000000,2012.000000,2.000000,0.000000,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,7500.25000,NaN,0.000000,2013.000000,4.000000,0.000000,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,10000.00000,6.000000,1.000000,2013.000000,34.000000,2.000000,5.00000,1984.000000,22943.212000,1.611964e+07,4.426344e+06,2012.000000,569032.313400,2.824619e+06,1.296269e+06,2.998083e+06,3.000000,2.000000,1.000000,1.000000


 ## Descriptive analysis ##
 
 With all the data available in the first database is extremely important to make a complete descriptive analysis of the variables to understand the type of information we are dealing with, which can give us an idea of which variables are relevants to solve our problem. 

For every single variable, we will run a single logistic regression to analyze its importance, and select only a subset of the features for our final model (there is a danger in this method: we may lose any effect due not to individual variables, but to the relationship between them)

First, we separate the data into train and test sets

In [100]:
db1_train, db1_test = train_test_split(db1.index, test_size=0.2, random_state=42)
scores = {}

### Price sensitivity ###

It indicates the client's sensitivity to the price: 1 (less sensitive) - 6 (more sensitive) 

The clients without data in the *Price Sensitivity* feature have a lower probability of buying the insurance, so we have included them in the analysis with the value 7, making them the more sensible to the price

In [173]:
db1.loc[np.isnan(db1['Price Sensitivity']), 'Price Sensitivity'] = 7
scores['Price Sensitivity'], fig = analyze_feature('Price Sensitivity', db1, db1_train, db1_test)
iplot(fig)

### PhoneType ###

Client's phone type: Fixed or Mobile

In [176]:
scores['PhoneType'], fig = analyze_feature('PhoneType', db1, db1_train, db1_test, categorical=True)
iplot(fig)